# Perlin noise

_Gilbert François Duivesteijn_


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# from IPython.core.display import display, HTML # my imports

In [ ]:
%matplotlib inline

## 1D Perlin noise

Exploration between 2 grid points.

In [ ]:
# Number of cells
nx = 4

# Random gradients at grid points with range [-1,1]
grad = 2*np.random.rand(nx+1)-1

In [ ]:
plt.figure(figsize=(7, 7))
plt.plot(grad, marker="o")
axs = plt.gca()
axs.spines['left'].set_position('zero')
axs.spines['right'].set_color('none')
axs.spines['bottom'].set_position('zero')
axs.spines['top'].set_color('none')
plt.title("Gradient values at grid points")
plt.xlabel("x")
plt.ylabel("v")
plt.show()
print("Grad values:")
print(grad)

In [ ]:
x = np.linspace(0, 1, 101)[:100]

ix0 = np.floor(x).astype("int")
ix1 = ix0 + 1

dx0 = x - ix0
dx1 = x - ix1

dg0 = grad[ix0]*dx0
dg1 = grad[ix1]*dx1

w = x - ix0

z_linear = (dg1 - dg0)*w + dg0
z_cubic  = (dg1 - dg0)*(3.0 - w * 2.0)*w**2 + dg0;

In [ ]:
plt.figure(figsize=(7, 7))
plt.plot(x, dg0, label="$dotgrad_0$")
plt.plot(x, dg1, label="$dotgrad_1$")
plt.plot(x, z_linear, label="$z_{linear}$")
plt.plot(x, z_cubic, label="$z_{smooth}$")

axs = plt.gca()
axs.spines['left'].set_position('zero')
axs.spines['right'].set_color('none')
axs.spines['bottom'].set_position('zero')
axs.spines['top'].set_color('none')
plt.legend()
plt.show()

## 1D Perlin noise functions

In [ ]:
def dotgrad(x, ix, grad):
    # distance vector
    dx = x - ix
    # grad times distance
    dz = grad[ix]*dx
    return dz

def lerp(a0, a1, w, interpolation):
    # Linear interpolation
    if interpolation == 0:
        return (a1 - a0)*w + a0
    # Cubic interpolation
    elif interpolation == 1:
        return (a1 - a0) * (3.0 - w * 2.0) * w * w + a0;
    else:
        raise ValueError(f"Invalid interpolation method: {interpolation}")
    
def perlin(x, interpolation):
    
    # Find closest surrounding grid points.
    ix0 = np.floor(x).astype("int")
    ix1 = ix0 + 1
    
    # Compute the value at the grid points, with dot product of grad*dist.
    z0 = dotgrad(x, ix0, grad)
    z1 = dotgrad(x, ix1, grad)
    
    # Interpolate between the values of the 2 surrounding grid points.
    z = lerp(z0, z1, x - ix0, interpolation)

    return z

In [ ]:
x = np.linspace(0, nx, nx*100+1)[:-1]

In [ ]:
z_linear = perlin(x, interpolation=0)
z_cubic = perlin(x, interpolation=1)

In [ ]:
plt.figure(figsize=(7, 7))
plt.plot(x, z_linear, label="$z_{linear}$")
plt.plot(x, z_cubic, label="$z_{cubic}$")

axs = plt.gca()
axs.spines['left'].set_position('zero')
axs.spines['right'].set_color('none')
axs.spines['bottom'].set_position('zero')
axs.spines['top'].set_color('none')
plt.legend()
plt.show()

## 2D Perlin noise functions

Following Ken Perlin's reference implementation [[1](https://mrl.cs.nyu.edu/~perlin/noise/)], this is a vectorized implementation using numpy.


In [ ]:
# Define number of grid cells
nx_cells = 4
ny_cells = 4

# Set grid indices
x = np.linspace(0, nx_cells, nx_cells+1)
y = np.linspace(0, ny_cells, ny_cells+1)
xx, yy = np.meshgrid(x, y)

# Create gradient vectors dvdx,dvdy for each grid point.
gradv = 2*np.random.rand(nx_cells+1, ny_cells+1,2)-1

In [ ]:
fig, ax = plt.subplots(1, 2, subplot_kw={"projection": "3d"}, figsize=(14, 7))
ax[0].plot_surface(xx, yy, gradv[:,:,0], cmap="inferno", linewidth=1, antialiased=True)
ax[1].plot_surface(xx, yy, gradv[:,:,1], cmap="inferno", linewidth=1, antialiased=True)
ax[0].set_zlim(-1, 1)
ax[0].set_title("$dv/dx$")
ax[0].set_xlabel("X")
ax[0].set_ylabel("Y")
ax[1].set_zlim(-1, 1)
ax[1].set_title("$dv/dy$")
ax[1].set_xlabel("X")
ax[1].set_ylabel("Y")
plt.show()

In [ ]:
def dotgrad(x, y, x0, y0, gradv):
    dx = x - x0
    dy = y - y0
    return (gradv[x0, y0, 0]*dx + gradv[x0, y0, 1]*dy)

def lerp(v0, v1, w, smooth=True):
    if smooth:
        v = (v1 - v0) * (3.0 - w * 2.0) * w * w + v0;
    else:
        v = (v1-v0)*w + v0
    return v

def perlin(x, y, smooth=True, debug=False):
    
    # Find closest gridpoints on left and right hand side of the point.
    x0 = np.floor(x).astype("int")
    x1 = x0 + 1
    y0 = np.floor(y).astype("int")
    y1 = y0 + 1
    
    # Compute the dot gradients for all gridpoints in the domain
    v00 = dotgrad(x, y, x0, y0, gradv)
    v01 = dotgrad(x, y, x1, y0, gradv)  
    v10 = dotgrad(x, y, x0, y1, gradv)
    v11 = dotgrad(x, y, x1, y1, gradv)
    
    v0x = lerp(v00, v01, x - x0, smooth)
    v1x = lerp(v10, v11, x - x0, smooth)
    vxy = lerp(v0x, v1x, y - y0, smooth)
    if debug:
        return vxy, v00, v01, v10, v11, v0x, v1x
    else:
        return vxy

In [ ]:
# Compute the noise within the domain

x = np.linspace(0, nx_cells, nx_cells*10+1)
y = np.linspace(0, ny_cells, ny_cells*10+1)
xx, yy = np.meshgrid(x, y)
xx = xx[:-1,:-1]
yy = yy[:-1,:-1]

zzl = perlin(xx, yy, smooth=False)
zzs = perlin(xx, yy, smooth=True)

In [ ]:
fig, ax = plt.subplots(1, 2, subplot_kw={"projection": "3d"}, figsize=(14, 7))
ax[0].plot_surface(xx, yy, zzl, cmap="inferno", linewidth=1, antialiased=True)
ax[0].set_zlim(-1, 1)
ax[0].set_title("$v_{linear}$")
ax[0].set_xlabel("X")
ax[0].set_ylabel("Y")
ax[1].plot_surface(xx, yy, zzs, cmap="inferno", linewidth=1, antialiased=True)
ax[1].set_zlim(-1, 1)
ax[1].set_title("$v_{smooth}$")
ax[1].set_xlabel("X")
ax[1].set_ylabel("Y")
plt.show()

## Unit test for MSX code

In [ ]:
# Define number of grid cells
nx_cells = 4
ny_cells = 4

# Offset
ox=0
oy=0

# n_pixels per cell
px = (256-2*ox)/nx_cells
py = (192-2*oy)/ny_cells

print(f"px: {px}")
print(f"py: {py}")

In [ ]:
# Screen pixels

x = np.arange(0, 256)
y = np.arange(0, 192)

assert x.shape[0] == 256
assert y.shape[0] == 192

print("x = ")
print(x)
print("y = ")
print(y)

In [ ]:
xp = (x-2*ox)/px
yp = (y-2*oy)/py

assert xp.shape[0] == 256
assert yp.shape[0] == 192

print("xp = ")
print(xp)
print("yp = ")
print(yp)

In [ ]:
# Set grid indices
x = np.linspace(0, nx_cells, nx_cells+1)
y = np.linspace(0, ny_cells, ny_cells+1)
xx, yy = np.meshgrid(x, y)

# Create gradient vectors dvdx,dvdy for each grid point.
gradv_x = np.cos(xx)*np.cos(yy)
gradv_y = np.sin(yy)*np.sin(xx)
gradv = np.stack([gradv_x, gradv_y], axis=2)

# gradv = 2*np.random.rand(nx_cells+1, ny_cells+1,2)-1
# gradv.shape

In [ ]:
fig, ax = plt.subplots(1, 2, subplot_kw={"projection": "3d"}, figsize=(14, 7))
ax[0].plot_surface(xx, yy, gradv[:,:,0], cmap="inferno", linewidth=1, antialiased=True)
ax[1].plot_surface(xx, yy, gradv[:,:,1], cmap="inferno", linewidth=1, antialiased=True)
ax[0].set_zlim(-1, 1)
ax[0].set_title("$dv/dx$")
ax[0].set_xlabel("X")
ax[0].set_ylabel("Y")
ax[1].set_zlim(-1, 1)
ax[1].set_title("$dv/dy$")
ax[1].set_xlabel("X")
ax[1].set_ylabel("Y")
plt.show()


In [ ]:
gradv[:,:,0]

In [ ]:
gradv[:,:,1]

In [ ]:
yy, xx = np.meshgrid(yp, xp)
print(xx.shape, yy.shape)


zzl, v00, v01, v10, v11, v0x, v1x = perlin(xx, yy, smooth=False, debug=True)
zzs, v00, v01, v10, v11, v0x, v1x = perlin(xx, yy, smooth=True, debug=True)

# zzs = perlin(xx, yy, smooth=True)

In [ ]:
_x=100
_y=10

print(v00[_x,_y])
print(v01[_x,_y])
print(v10[_x,_y])
print(v11[_x,_y])
print(v0x[_x,_y])
print(v1x[_x,_y])
print(zzl[_x,_y])
print(zzl.shape)
print(xp.shape)

In [ ]:
fig, ax = plt.subplots(1, 2, subplot_kw={"projection": "3d"}, figsize=(14, 7))
ax[0].plot_surface(xx, yy, zzl, cmap="inferno", linewidth=1, antialiased=True)
ax[0].set_zlim(-1, 1)
ax[0].set_title("$v_{linear}$")
ax[0].set_xlabel("X")
ax[0].set_ylabel("Y")
ax[1].plot_surface(xx, yy, zzs, cmap="inferno", linewidth=1, antialiased=True)
ax[1].set_zlim(-1, 1)
ax[1].set_title("$v_{smooth}$")
ax[1].set_xlabel("X")
ax[1].set_ylabel("Y")
plt.show()

In [ ]:
plt.figure()
plt.plot(zzl[96,:])
plt.show()